In [2]:
from pyspark.sql import SparkSession
from datetime import datetime, date
from pyspark.sql import Row
import pandas as pd
import os 
import sys 

os.environ['PYSPARK_DRIVER_PYTHON_OPTS']= "notebook"
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
os.environ['PYSPARK_PYTHON'] = sys.executable

In [3]:
spark = SparkSession.builder.getOrCreate()
spark

In [4]:
df = pd.read_csv('../data/sondaggio_sostenibilità_unipa.csv')
df.head()

# drop first test answer and last unused column
df.drop(0, inplace=True)
df.drop("Unnamed: 81", axis=1, inplace=True)

df.head(5)

,Informazioni cronologiche,Ho letto e accettato l'informativa e confermo inoltre di avere più di 18 anni,Quanti anni hai?,Genere,Da quante persone è composto il tuo nucleo familiare?,Occupazione,Quanto è grande la tua azienda?,Da che regione provieni?,Provincia di provenienza,In che regione lavori/studi?,...,Rifiuti - Lavoro o Università [C'è un'attenzione nel ridurre l'uso della plasica],Rifiuti - Lavoro o Università [C'è un'attenzione alla gestione di materiali organici],Rifiuti - Lavoro o Università [C'è un'attenzione alla gestione di materiali inorganici],"Rifiuti - Casa [Vengono implementate le 3R: Riduzione, Riuso e Riciclo]",Rifiuti - Casa [C'è un'attenzione nel ridurre l'uso della plasica],Rifiuti - Casa [C'è un'attenzione alla gestione di materiali organici],Rifiuti - Casa [C'è un'attenzione alla gestione di materiali inorganici],"Inserire il numero di eventi (conferenze, workshop, sensibilizzazione, formazione pratica, festival, etc.) legati ai problemi di sostenibilità e dell'ambiente organizzati dalla propria azienda e/o università (media per anno degli ultimi 3 anni). \nSelezionare una delle seguenti opzioni:","Inserire il numero totale di attività organizzate dalle organizzazioni studentesche o da eventuali organizzazioni interne al proprio ambiente di lavoro per anno. Ad esempio, seminari, webinars, formazione, eventi sportivi, bazaar di materiali riciclati, comunità di divulgazione, etc.. \nSelezionare una delle seguenti opzioni:","La presenza di strutture ""green"" all'interno del proprio campus e/o della propria azienda che siano accessibili al pubblico, per esempio durante attività culturali, indicano un impatto più ampio di tali strutture nei loro dintorni (Festival Culturali, teatro, performance musicali, esibizioni, attività virtuali, etc.).\nSelezionare una delle seguenti opzioni:"
1,2023/06/18 1:13:56 PM EEST,Sì,Tra 26 e 50 anni,Femminile,4,Non impiegato,Da 51 a 250 impiegati (media impresa),Sicilia,Messina,Sicilia,...,1-50%,1-50%,51-75%,In Progetto,In Progetto,1-50%,In Progetto,1 - 4,3 - 4,1 evento per anno
2,2023/06/18 2:46:14 PM EEST,Sì,Maggiore di 50 anni,Femminile,4,Lavoratore presso azienda pubblica,Da 51 a 250 impiegati (media impresa),Sicilia,Messina,Sicilia,...,76-100%,76-100%,76-100%,76-100%,76-100%,76-100%,76-100%,1 - 4,1 - 2,3 eventi per anno
3,2023/06/18 3:09:21 PM EEST,Sì,Tra 26 e 50 anni,Maschile,3,Lavoratore presso azienda privata,Da 1 a 10 impiegati (micro impresa),Sicilia,Messina,Sicilia,...,1-50%,1-50%,1-50%,1-50%,1-50%,1-50%,1-50%,0,0,Nessuna
4,2023/06/18 4:17:34 PM EEST,Sì,Tra 18 e 25 anni,Femminile,4,Studente Lavoratore,Da 1001 a 10000 impiegati,Sicilia,Messina,Sicilia,...,76-100%,76-100%,76-100%,76-100%,76-100%,76-100%,76-100%,5 - 17,1 - 2,2 eventi per anno
5,2023/06/18 4:46:22 PM EEST,Sì,Tra 26 e 50 anni,Femminile,4,Non impiegato,Da 1001 a 10000 impiegati,Sicilia,Messina,Sicilia,...,No,No,No,1-50%,In Progetto,1-50%,1-50%,0,0,Nessuna


In [5]:
df = spark.createDataFrame(df)

In [6]:
df.show(1)

+-------------------------+-----------------------------------------------------------------------------+----------------+---------+-----------------------------------------------------+-------------+-------------------------------+------------------------+------------------------+----------------------------+------------------------------------+------------------+--------------------+---------------------------------+-------------------------------+--------------------------------+-----------------------------------------+-----------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---

In [12]:
# df.select('*').take(1)[0].asDict()
# df.select('*').take()
rows = df.collect()

In [37]:
import json

questions_dir = '../data/mobility_questions.json'
with open(questions_dir, encoding='utf-8') as answers:
    answers = json.load(answers)['questions']

questions = [answers[a]['question_text'] for a in answers]

questions_map = {a:k for (a, k) in zip(questions, answers.keys())}
questions_map

{'Nella tua giornata tipo, a che ora esci di casa?': 'm_0',
 'Qual è il veicolo che usi quotidianamente?': 'm_1',
 'Cosa alimenta il tuo veicolo?': 'm_2',
 "In quale classe ambientale si trova il veicolo che usi quotidianamente? (basati sull'anno di immatricolazione se non conosci la classe)": 'm_3',
 'Quante volte a settimana esci di casa?': 'm_4',
 'Di queste volte, quanto spesso usi il mezzo di trasporto che hai specificato poco sopra?': 'm_5',
 'Quanto traffico incontri quando esci di casa?': 'm_6',
 'Quanti minuti impieghi solitamente per trovare parcheggio per il tuo veicolo?': 'm_7',
 'Indicativamente, quanto è grande in m² la tua residenza?': 'm_8',
 'Indicativamente, quanti m² di spazi verdi sono presenti nella tua residenza?': 'm_9',
 'Indicativamente, che percentuale della tua residenza è riservata al parcheggio?': 'm_10',
 'Quanto spendi in carburante ogni mese?': 'm_11',
 "Quanto spendi in trasporti ecosostenibili ogni mese? (per esempio, caricando l'auto elettrica)": 'm_1

In [22]:
r = rows[0].asDict()

for k in r.keys():
    print(f"{k}: {r[k]}")
    print(k, r[k])
    print()

Informazioni cronologiche: 2023/06/18 1:13:56 PM EEST
Informazioni cronologiche 2023/06/18 1:13:56 PM EEST

Ho letto e accettato l'informativa e confermo inoltre di avere più di 18 anni: Sì
Ho letto e accettato l'informativa e confermo inoltre di avere più di 18 anni Sì

Quanti anni hai?: Tra 26 e 50 anni
Quanti anni hai? Tra 26 e 50 anni

Genere: Femminile
Genere Femminile

Da quante persone è composto il tuo nucleo familiare?: 4
Da quante persone è composto il tuo nucleo familiare? 4

Occupazione: Non impiegato
Occupazione Non impiegato

Quanto è grande la tua azienda?: Da 51 a 250 impiegati (media impresa)
Quanto è grande la tua azienda? Da 51 a 250 impiegati (media impresa)

Da che regione provieni?: Sicilia
Da che regione provieni? Sicilia

Provincia di provenienza: Messina
Provincia di provenienza Messina

In che regione lavori/studi?: Sicilia
In che regione lavori/studi? Sicilia

Provincia del luogo di lavoro/studio: Messina
Provincia del luogo di lavoro/studio Messina

Invalidi